In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import sys
from modeling.make_mini_model import *

In [2]:
spot = pd.read_excel('../최종DB관리/Spot_VSS/2019.02.13.최종_통합DB2.xlsx')

In [4]:
# Remove operation that CoreML doesn't supported
# Make mini graph
tf.reset_default_graph()
ids = spot["ID2"].astype(int)#np.float32
tag = spot["Tag_encoder"].astype(int)
city = spot["City_encoder"].astype(int)
state = spot["State_encoder"].astype(int)
embedding=Resave_model(spot,['ID2',"City_encoder"],ID=ids,tag=tag,city=city,states=state,
             ckpt_dir = 'ckpt_model',
             rnn_size=[4000,150],
             item_size=[len(spot),spot.City_encoder.nunique()],
             model_number =8
            )

INFO:tensorflow:Restoring parameters from ckpt_model/attribute_model-8
INFO:tensorflow:Restoring parameters from ckpt_model/attribute_model-8
ckpt_model/custom_model 로 모델 저장 완료 및 embedding input 복원!


In [6]:
from modeling.tensor_conver_to_pb import freeze_graph 
import pickle
import tfcoreml

In [7]:
# embedding 가중치 따로 저장
with open('embedding_final.pkl',"wb") as f:
    pickle.dump(embedding,f)    

In [8]:
freeze_graph(model_dir="ckpt_model/" ,output_node_names="result2,ID,category,states,gru_layer/add_6") # ,rnn_state_result,

ckpt_model/custom_model
INFO:tensorflow:Restoring parameters from ckpt_model/custom_model
INFO:tensorflow:Froze 9 variables.
Converted 9 variables to const ops.
57 ops in the final graph.


In [9]:
#tensor name and result checking
tf.reset_default_graph()
with tf.gfile.GFile('ckpt_model/frozen_model.pb', "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    g_in = tf.import_graph_def(graph_def, name="")
sess = tf.Session(graph=g_in)
name=[n.name for n in tf.get_default_graph().as_graph_def().node  ]
for i in name:
    print(i)
#
IDs,states,category,result2,state = sess.run(["ID:0","states:0","category:0","result2:0",'gru_layer/add_6:0'],feed_dict={"input:0":embedding[[270]],"rnn_state:0":np.zeros((1,4150))})

input
rnn_state
gru_layer/Const
gru_layer/Const_1
gru_layer/input_/reset_Weigth
gru_layer/input_/reset_Weigth/read
gru_layer/input_/Update_Weigth
gru_layer/input_/Update_Weigth/read
gru_layer/input_/hidden_Weigth
gru_layer/input_/hidden_Weigth/read
gru_layer/state_/reset_Weigth
gru_layer/state_/reset_Weigth/read
gru_layer/state_/update_Weigth
gru_layer/state_/update_Weigth/read
gru_layer/state_/hidden_Weigth
gru_layer/state_/hidden_Weigth/read
gru_layer/bias_/reset_bias
gru_layer/bias_/reset_bias/read
gru_layer/bias_/update_bias
gru_layer/bias_/update_bias/read
gru_layer/bias_/hidden_bias
gru_layer/bias_/hidden_bias/read
gru_layer/MatMul
gru_layer/MatMul_1
gru_layer/add
gru_layer/add_1
gru_layer/Sigmoid
gru_layer/MatMul_2
gru_layer/MatMul_3
gru_layer/add_2
gru_layer/add_3
gru_layer/Sigmoid_1
gru_layer/MatMul_4
gru_layer/Mul
gru_layer/MatMul_5
gru_layer/add_4
gru_layer/add_5
gru_layer/Tanh
gru_layer/sub/x
gru_layer/sub
gru_layer/Mul_1
gru_layer/Mul_2
gru_layer/add_6
MatMul
result2
Const

In [11]:
# result check
check=pd.DataFrame(columns=['rate','ID',"category"])
check['rate'] = result2.reshape(1333)
check['ID'] = IDs.reshape(1333)
check['category'] = category.reshape(1333)
check["states"] = states.reshape(1333)

In [12]:
check2=check.sort_values(['rate'],ascending=False)
check2

,rate,ID,category,states
662,6.782130,662,4,4
65,5.298802,65,3,4
94,5.177513,94,3,4
87,5.089226,87,3,4
105,5.044178,105,3,4
14,4.892836,14,3,4
1040,4.865443,1040,5,4
11,4.851724,11,3,4
129,4.782138,129,3,4
1033,4.683892,1033,3,4


In [13]:
print(IDs)
print(category)
print(result2) # 결과 안달라지는지 확인
print(state)
print(states)

[[   0    1    2 ... 1330 1331 1332]]
[[3 3 3 ... 0 0 4]]
[[ 0.73262197  0.16956767  3.1343973  ... -0.01788715  0.02142866
   0.81093425]]
[[ 0.37799266  0.09162887  0.06613003 ... -0.09269695  0.17587723
  -0.2125981 ]]
[[4 4 4 ... 2 1 4]]


In [14]:
# convert pb to coreml
# You must match input shape
# ID와 category 결과 함께 내보내기
tfcoreml.convert(tf_model_path = 'ckpt_model/frozen_model.pb',mlmodel_path = 'ckpt_model/my_model.mlmodel',
                     output_feature_names = ['result2:0','ID:0','category:0','states:0',"gru_layer/add_6:0"],
                     input_name_shape_dict = {"input/placeholder:0": [1,4150]  ,"rnn_state/placeholder:0":[1,4150]})


Loading the TF graph...
Graph Loaded.
Done.
Now finding ops in the TF graph that can be dropped for inference
Now starting translation to CoreML graph.
1/57: Analysing op name: Const_3 ( type:  Const )
2/57: Analysing op name: Const_1 ( type:  Const )
3/57: Analysing op name: Const ( type:  Const )
4/57: Analysing op name: gru_layer/sub/x ( type:  Const )
5/57: Analysing op name: gru_layer/bias_/hidden_bias ( type:  Const )
6/57: Analysing op name: gru_layer/bias_/hidden_bias/read ( type:  Identity )
7/57: Analysing op name: gru_layer/bias_/update_bias ( type:  Const )
8/57: Analysing op name: gru_layer/bias_/update_bias/read ( type:  Identity )
9/57: Analysing op name: gru_layer/bias_/reset_bias ( type:  Const )
10/57: Analysing op name: gru_layer/bias_/reset_bias/read ( type:  Identity )
11/57: Analysing op name: gru_layer/state_/hidden_Weigth ( type:  Const )
12/57: Analysing op name: gru_layer/state_/hidden_Weigth/read ( type:  Identity )
13/57: Analysing op name: gru_layer/state_

input {
  name: "rnn_state__0"
  type {
    multiArrayType {
      shape: 4150
      dataType: DOUBLE
    }
  }
}
input {
  name: "input__0"
  type {
    multiArrayType {
      shape: 4150
      dataType: DOUBLE
    }
  }
}
output {
  name: "result2__0"
  type {
    multiArrayType {
      shape: 1333
      dataType: DOUBLE
    }
  }
}
output {
  name: "states__0"
  type {
    multiArrayType {
      shape: 1333
      dataType: DOUBLE
    }
  }
}
output {
  name: "category__0"
  type {
    multiArrayType {
      shape: 1333
      dataType: DOUBLE
    }
  }
}
output {
  name: "ID__0"
  type {
    multiArrayType {
      shape: 1333
      dataType: DOUBLE
    }
  }
}
output {
  name: "gru_layer__add_6__0"
  type {
    multiArrayType {
      shape: 4150
      dataType: DOUBLE
    }
  }
}